In [40]:
UID_no = 118406473                                     
Last_Name = "Kanna"
First_Name = "Suprajha"

import random
import hmac
import hashlib

def xor(byteseq1, byteseq2):
  #print("value of byteseq1",byteseq1)
  #print("value of byteseq1",byteseq2)
  l1 = [b for b in byteseq1]   # Splits input byte sequences into byte size integers
  l2 = [b for b in byteseq2]   # Splits input byte sequences into byte size integers
  l1joinl2 = zip(l1,l2)
  l1xorl2 = [bytes([elem1^elem2]) for elem1,elem2 in l1joinl2]     # XOR ^ operator does the operation on interger values and zip is a list
  result = b''.join(l1xorl2)  
  return result
    #result = xor(b1,b2)
#print(b1, ' xor ',b2,' = ',result)

def feistel_block(lefthalf_input, righthalf_input, k):            #contains left, right half of feistel structure input along with key as parameters
  lefthalf_output = righthalf_input
  #print("k val..",k)
  #print(type(k))
  byteval = b''.join(k)
  righthalf_xor = F(righthalf_input,byteval,8) #traverses in the string
  righthalf_output = xor(lefthalf_input,righthalf_xor)
  return lefthalf_output, righthalf_output

def F(byteseq, k, outputlen):
  h = hmac.new(bytes(k), byteseq, hashlib.sha1) #we use hmac hash and it returns the first outputlen bytes of the hash value
  return h.digest()[:outputlen]
#foutput = F(b'\x23', b'\x45')
#print(foutput)
# In a real Feistel implementation, different keys are used in different rounds. Here
# we use 64bit keys so for 16 rounds, we need 16 random 8byte keys. We can just generate
# 16 random 8 byte numbers we use the random.randint() function to be able to set the seed
# value and create the same keys for both the encoder and the decoder

def gen_keylist(keylenbytes, numkeys, seed):    #this function generates the keys required to perform the encryption and decryption for all rounds using the seed key
    keylist = []    # keylist = [numkeys elements of 'bytes' type and keylenbytes bytes long each]
    random.seed(seed)   
    for i in range(numkeys):
      bytelist = bytes([random.randint(0,255)])
      keylist.append(bytelist)   # example [b'\x12\xa4\x2f\xb4\x67\x80\xd4\x12', and 15 more if numkeys is 16]
    return keylist
# Use the random.randint(min,max) function to generate individual
    # random integers in range [min, max]. Generate a list of numkeys
    # random byte sequences each of them keylenbytes bytes long to be used as 
    # keys for numkeys stages of the feistel encoder. To make sure we have control over
    # the generated random numbers meaning that the same sequence is 
    # generated in different runs of our program,  

def feistel_enc(inputblock, num_rounds, masterkey): 
   #this func takes input blocks, encrypts using Fiestel algorithm and returns CT
  keylist = gen_keylist(8, num_rounds, masterkey)   # implement num_rounds of calling the block function
    # Inputs:
    # input sequence: byte sequence representing input block
    # num_rounds: integer representing number of rounds in the feistel 
    # seed: integer to set the random number generator to defined state
     # Output:
    # cipherblock: byte sequence
    # first generate the required keys
  left_inputblock = inputblock[0:8]
  right_inputblock = inputblock[8:16]

  if(len(left_inputblock)<8):
    val = 8 - len(left_inputblock)
    for i in range(val):
      left_inputblock = left_inputblock + b'\x20'

  if(len(right_inputblock)<8):
    val = 8 - len(right_inputblock)    
    for i in range(val):
      right_inputblock = right_inputblock + b'\x20'

  for i in range(0,16):
    left_inputblock, right_inputblock = feistel_block(left_inputblock, right_inputblock, keylist[i])


  cipherblock = right_inputblock + left_inputblock
  return cipherblock

def feistel_dec(inputblock, num_rounds, seed): # Make sure you use the keys in reverse order during decryption
  keylist = gen_keylist(8, num_rounds, seed) # apply the num_rounds times of the block function
  left_inputblock = inputblock[0:8]
  right_inputblock = inputblock[8:16]

  for i in range(0,16):
      left_inputblock, right_inputblock = feistel_block(left_inputblock, right_inputblock, keylist[15-i]) #the key starts from n-1

  plainblock = right_inputblock + left_inputblock
  return plainblock

#def main():
   # feistel_enc_test("test.txt", 8, 16, "test_enc.txt")
   # feistel_dec_test("test_enc.txt",8,16, "test_dec.txt")

if __name__ == "__main__":
    encryptnstring_val = feistel_enc(b'isthis16bytes?', 16, 50)
    print("cipher text is..",encryptnstring_val)
    decryptdstring_val = feistel_dec(b'}\xd9\x93-G\x8e\xaa5\x95\x84\n\xb7q\xc4>\xb6', 16, 50)
    print("decrypted value..",decryptdstring_val)

TypeError: ignored